In [2]:
from pathlib import Path
from openai import OpenAI
 
system_promtp = """
你是一位专注于图片内容提取的专业人士, 你需要帮助人类把图片内容提取出来保存为字典格式数据。
你只能输出以下DataFrame格式, 内容只包含下列字段, 禁止输出其他内容, 禁止输出字典以外的文字信息: 
选项1: 
{  
    "日期"：list,
    "序号": list,  
    "证券代码": list,  
    "证券名称": list,
    "总持仓" :list,
    "可变持仓":list,
    "参考市值":list,
    "现价":list,
    "涨幅":list,
    "当日盈亏":list,
}  
"""
user = """
将2-8页整理为dataframe格式数据。
"""

client = OpenAI(
        api_key="sk-8rz6aB58IUEYFyN8kLwyNk1FD0zl9HCjMTzJOxxWJ2RY9h0q",
        base_url="https://api.moonshot.cn/v1",
    )
 
# moonshot.pdf 是一个示例文件, 我们支持文本文件和图片文件，对于图片文件，我们提供了 OCR 的能力
# 上传文件时，我们可以直接使用 openai 库的文件上传 API，使用标准库 pathlib 中的 Path 构造文件
# 对象，并将其传入 file 参数即可，同时将 purpose 参数设置为 file-extract；注意，目前文件上传
# 接口仅支持 file-extract 一种 purpose 值。
file_object = client.files.create(file=Path("水星基金-持仓截图250828.pdf"), purpose="file-extract")
 
# 获取结果
# file_content = client.files.retrieve_content(file_id=file_object.id)
# 注意，某些旧版本示例中的 retrieve_content API 在最新版本标记了 warning, 可以用下面这行代替
# （如果使用旧版本的 SDK，可以继续延用 retrieve_content API）
file_content = client.files.content(file_id=file_object.id).text
 
# 把文件内容通过系统提示词 system prompt 放进请求中
messages = [
    {
        "role": "system",
        "content": system_promtp,
    },
    {
        "role": "system",
        "content": file_content, # <-- 这里，我们将抽取后的文件内容（注意是文件内容，而不是文件 ID）放置在请求中
    },
    {"role": "user", "content": user},
]
 
# 然后调用 chat-completion, 获取 Kimi 的回答
completion = client.chat.completions.create(
  model="kimi-k2-0905-preview",
  messages=messages,
  temperature=0.6,
)
 
print(completion.choices[0].message)

ChatCompletionMessage(content='{"日期":["2025-07-07","2025-07-07"],"序号":[1,2],"证券代码":["204001","002636"],"证券名称":["GC001","金安国纪"],"总持仓":[46150,0],"可变持仓":[0,0],"参考市值":[4615000.00,0.00],"现价":[100.000,12.47],"涨幅":["","9.96%"],"当日盈亏":["",32106.00]}', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)
